In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/men-women-classification/men/00002129.jpg
/kaggle/input/men-women-classification/men/00001981.jpg
/kaggle/input/men-women-classification/men/00001965.jpg
/kaggle/input/men-women-classification/men/00001915.jpg
/kaggle/input/men-women-classification/men/00000399.jpg
/kaggle/input/men-women-classification/men/00000390.jpg
/kaggle/input/men-women-classification/men/07 (4).jpg
/kaggle/input/men-women-classification/men/00000797.jpg
/kaggle/input/men-women-classification/men/00001179.jpg
/kaggle/input/men-women-classification/men/00000030.jpg
/kaggle/input/men-women-classification/men/00000848.jpg
/kaggle/input/men-women-classification/men/00000042.jpg
/kaggle/input/men-women-classification/men/00000127.jpg
/kaggle/input/men-women-classification/men/00000113.jpg
/kaggle/input/men-women-classification/men/00002266.jpg
/kaggle/input/men-women-classification/men/00001267.jpeg
/kaggle/input/men-women-classification/men/00001144.jpg
/kaggle/input/men-women-classification/men/000011

In [6]:
import os
import random
import shutil
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop

In [7]:
!cp -r /kaggle/input/men-women-classification/men /kaggle/working
!cp -r /kaggle/input/men-women-classification/women /kaggle/working

In [8]:
!mkdir /kaggle/working/train
!mkdir /kaggle/working/valid

In [ ]:
!mkdir /kaggle/working/valid/men
!mkdir /kaggle/working/valid/women

In [ ]:
!ls /kaggle/working/valid/

In [ ]:
def move_random_files(source_dir, destination_dir, percentage):
    
    # Get a list of all files in the source directory
    file_list = os.listdir(source_dir)

    # Calculate the number of files to move based on the given percentage
    num_files = int(len(file_list) * percentage)

    # Randomly select the files to move
    files_to_move = random.sample(file_list, num_files)

    # Move the selected files to the destination directory
    for file_name in files_to_move:
        source_file = os.path.join(source_dir, file_name)
        destination_file = os.path.join(destination_dir, file_name)
        shutil.move(source_file, destination_file)

In [ ]:
#Men
men_source_directory = '/kaggle/working/men'
men_validation_directory = '/kaggle/working/valid/men'
percentage_to_move = 0.2  # 20% of the files
move_random_files(men_source_directory, men_validation_directory, percentage_to_move)

In [ ]:
#Women
women_source_directory = '/kaggle/working/women'
women_validation_directory = '/kaggle/working/valid/women'
percentage_to_move = 0.2  # 20% of the files
move_random_files(women_source_directory, women_validation_directory, percentage_to_move)

In [ ]:
!ls /kaggle/working/valid/

In [ ]:
print(len(os.listdir('/kaggle/working/men')))
print(len(os.listdir('/kaggle/working/valid/men')))


In [ ]:
print(len(os.listdir('/kaggle/working/women')))
print(len(os.listdir('/kaggle/working/valid/women')))

In [ ]:
print(len(os.listdir('/kaggle/input/men-women-classification/men')))
print(len(os.listdir('/kaggle/input/men-women-classification/women')))

In [ ]:
!mv  /kaggle/working/men /kaggle/working/train/men
!mv  /kaggle/working/women /kaggle/working/train/women

In [ ]:
!ls /kaggle/working/train/men

In [ ]:
# Define our example directories and files
base_dir = '/kaggle/working/'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'valid')

# Directories with training pictures
men_trian_dir = os.path.join(train_dir,'men')
women_train_dir = os.path.join(train_dir,'women')

# Directory with validation cat pictures
men_validation_dir = os.path.join(validation_dir,'men')
women_validation_dir = os.path.join(validation_dir,'women')

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    class_mode='binary',
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers

pre_trained_model = VGG16(input_shape = (150, 150, 3),
                                weights='/kaggle/input/kernels-notop-h5/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                include_top = False)

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
pre_trained_model.summary()

In [ ]:
last_layer = pre_trained_model.get_layer('block5_conv2')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                        # This Has Regularization effects                 
# Add a final softmax layer for classification
x = layers.Dense(1, activation='sigmoid')(x)           

# Append the dense network to the base model
model = Model(pre_trained_model.input, x) 

# Print the model summary. See your dense network connected at the end.
model.summary()

In [ ]:
model.compile(optimizer = RMSprop(learning_rate=0.001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
# Train the model.
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 50,
            epochs = 20,
            verbose = 1)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()


In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers

pre_trained_model = InceptionV3(input_shape = (150, 150, 3),
                                weights='/kaggle/input/interception/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                include_top = False)

In [ ]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [ ]:
last_layer = pre_trained_model.get_layer('mixed8')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                        # This Has Regularization effects                 
# Add a final softmax layer for classification
x = layers.Dense(1, activation='sigmoid')(x)           

# Append the dense network to the base model
model = Model(pre_trained_model.input, x) 

# Print the model summary. See your dense network connected at the end.
model.summary()

In [ ]:
model.compile(optimizer = RMSprop(learning_rate=0.001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
# Train the model.
history_Inception = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 50,
            epochs = 20,
            verbose = 1)

In [4]:
acc = history_Inception.history['accuracy']
val_acc = history_Inception.history['val_accuracy']
loss = history_Inception.history['loss']
val_loss = history_Inception.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()


NameError: name 'history_Inception' is not defined

In [3]:
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

# Path to your image
img_path = '/kaggle/input/ofirphoto/WhatsApp Image 2024-08-10 at 01.01.35_dee7aaa0.jpg'

# Load and preprocess the image
test_image = load_and_preprocess_image(img_path)

# Make a prediction
prediction = model.predict(test_image)

# Interpret the result
prediction_label = "Woman" if prediction[0] > 0.5 else "Man"
print(f"Prediction: {prediction_label}")

# Display the image with the prediction
img = image.load_img(img_path, target_size=(150, 150))
plt.imshow(img)
plt.title(f"Prediction: {prediction_label}")
plt.axis('off')
plt.show()


NameError: name 'model' is not defined